In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Activation, Flatten, Dense, Dropout, GlobalAveragePooling2D
import cv2

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
print("x_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("x_test shape:", X_test.shape, "y_test shape:", y_test.shape)

x_train shape: (60000, 28, 28) y_train shape: (60000,)
x_test shape: (10000, 28, 28) y_test shape: (10000,)


In [ ]:
def reshape(X, size_x, size_y):
  X_reshaped = []
  for image in X:
    resized_image = cv2.resize(image, (size_x,size_y))
    X_reshaped.append(cv2.merge((resized_image, resized_image, resized_image)))
  return np.asarray(X_reshaped).astype('float32') / 255

In [ ]:
w, h = 75,75

X_train = reshape(X_train, w, h)
X_test = reshape(X_test, w, h)

In [ ]:
print("x_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("x_test shape:", X_test.shape, "y_test shape:", y_test.shape)

x_train shape: (60000, 75, 75, 3) y_train shape: (60000,)
x_test shape: (10000, 75, 75, 3) y_test shape: (10000,)


In [ ]:
X_train = X_train.reshape(X_train.shape[0], w, h, 3)
X_test = X_test.reshape(X_test.shape[0], w, h, 3)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print(X_train.shape[0], 'train set')
print(X_test.shape[0], 'test set')


60000 train set
10000 test set


In [ ]:
base_model = tf.keras.applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=X_train[0].shape)
base_model.trainable = False

add_model = tf.keras.Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(tf.keras.layers.Dense(2048, activation='relu'))
add_model.add(tf.keras.layers.Dense(500, activation='relu'))
add_model.add(Dense(10, activation='softmax'))

model = add_model
model.compile(loss='categorical_crossentropy', 
              optimizer="adam",
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 1, 1, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              2049000   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10010     
Total params: 28,058,146
Trainable params: 6,255,362
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
model.fit(np.array(X_train),
         np.array(y_train),
         batch_size=64,
         epochs=10,
         validation_data=(np.array(X_test), np.array(y_test)))

Epoch 1/10
938/938 [==============================] - 594s 629ms/step - loss: 0.6889 - accuracy: 0.7729 - val_loss: 0.4445 - val_accuracy: 0.8357
Epoch 2/10
938/938 [==============================] - 593s 632ms/step - loss: 0.3805 - accuracy: 0.8600 - val_loss: 0.4030 - val_accuracy: 0.8513
Epoch 3/10
938/938 [==============================] - 596s 635ms/step - loss: 0.3170 - accuracy: 0.8808 - val_loss: 0.4233 - val_accuracy: 0.8526
Epoch 4/10
938/938 [==============================] - 592s 631ms/step - loss: 0.2698 - accuracy: 0.8990 - val_loss: 0.4412 - val_accuracy: 0.8566
Epoch 5/10
938/938 [==============================] - 605s 645ms/step - loss: 0.2321 - accuracy: 0.9121 - val_loss: 0.4743 - val_accuracy: 0.8593
Epoch 6/10
938/938 [==============================] - 606s 647ms/step - loss: 0.1946 - accuracy: 0.9249 - val_loss: 0.4800 - val_accuracy: 0.8595
Epoch 7/10
938/938 [==============================] - 610s 650ms/step - loss: 0.1679 - accuracy: 0.9361 - val_loss: 0.5375 -